## Use Cohere

### Setup

In [ ]:
import os
from dotenv import load_dotenv
from llama_index.llms.cohere import Cohere
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Settings,
    PromptTemplate
)

load_dotenv()
api_key = os.getenv("API_KEY")

### Use cohere instead of OpenAI

In [ ]:
# Use cohere
cohere = Cohere(api_key=api_key)
cohere_embedding = CohereEmbedding(cohere_api_key=api_key)
Settings.llm = cohere
Settings.embed_model = cohere_embedding

### Load data and build an index

In [ ]:
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents=documents)

### Query your data

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author growing up?")
print(response)

### Viewing Queries and Events Using Logging

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
index.storage_context.persist()

### Retrieval-Augmented Generation

In [ ]:
response = cohere.complete("What did the author growing up?")
print(response)

In [ ]:
template = """
    We have provided context information below.
    --------------------
    {context}
    --------------------
    Given this information, please answer the question: {question}
"""
prompt_template = PromptTemplate(template=template)

question = "What did the author growing up?"
context = query_engine.query(question)

prompt = prompt_template.format(context=context, question=question)
response = cohere.complete(prompt, formatted=True)

print(response)